In [1]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Import and Authenticate

In [3]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import datetime
import os
import json

# --- User parameters ---
# Create a folder for data
DATA_DIR = "sentinel_downloads"
os.makedirs(DATA_DIR, exist_ok=True)

# Your Copernicus Data Space credentials
CATALOGUE_URL = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products"
USER = "ravik2.paschim@gmail.com"  # <-- put your Copernicus Data Space username
PASSWORD = "XHM0UExR>W36"  # <-- put your password

# Connect to the new Copernicus Data Space API
api = SentinelAPI(USER, PASSWORD, "https://catalogue.dataspace.copernicus.eu/apihub")

print(" Connected to Copernicus Data Space successfully!")


 Connected to Copernicus Data Space successfully!


In [13]:
# Load credentials, AOI path, and time range from config.yaml
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

USERNAME = config["username"]
PASSWORD = config["password"]
AOI_PATH = config["aoi_path"]
START_DATE = config["start_date"]
END_DATE = config["end_date"]

print("Configuration file loaded successfully.")


Configuration file loaded successfully.


In [14]:
import yaml
from urllib.parse import quote

# ===================== LOAD CONFIGURATION =====================
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

USERNAME = config["username"]
PASSWORD = config["password"]
AOI_PATH = config["aoi_path"]
START_DATE = config["start_date"]
END_DATE = config["end_date"]

# ===================== AUTHENTICATION =====================
print("Authenticating...")
token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
data = {
    "client_id": "cdse-public",
    "grant_type": "password",
    "username": USERNAME,
    "password": PASSWORD,
}
resp = requests.post(token_url, data=data)
resp.raise_for_status()
access_token = resp.json()["access_token"]
headers = {"Authorization": f"Bearer {access_token}"}
print("Auth success!")

# ===================== LOAD AOI GEOJSON =====================
aoi = gpd.read_file(AOI_PATH)
geom = aoi.geometry.iloc[0]
try:
    polygon_wkt = geom.to_wkt()
except AttributeError:
    polygon_wkt = geom.wkt
print("AOI loaded successfully!")

# Encode polygon for URL safety (avoid 400 Bad Request)
polygon_encoded = quote(polygon_wkt)

# ===================== QUERY SENTINEL-1 PRODUCTS =====================
query_url = (
    "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?"
    "$filter="
    f"OData.CSC.Intersects(area=geography'SRID=4326;{polygon_encoded}') "
    "and Collection/Name eq 'SENTINEL-1' "
    f"and ContentDate/Start gt {START_DATE} "
    f"and ContentDate/Start lt {END_DATE}"
    "&$orderby=ContentDate/Start desc"
    "&$top=20"
)

print("Sending query to Copernicus Data Space...")
resp = requests.get(query_url, headers=headers)
resp.raise_for_status()

# ===================== PROCESS RESULTS =====================
results = resp.json().get("value", [])
print(f"Found {len(results)} Sentinel-1 products\n")

for r in results:
    print(f"{r['Name']} — {r['ContentDate']['Start']}")


Authenticating...
Auth success!
AOI loaded successfully!
Sending query to Copernicus Data Space...
Found 20 Sentinel-1 products

S1A_IW_GRDH_1SDV_20250923T005158_20250923T005223_061108_079DF5_7688_COG.SAFE — 2025-09-23T00:51:58.781428Z
S1A_IW_GRDH_1SDV_20250923T005158_20250923T005223_061108_079DF5_250B.SAFE — 2025-09-23T00:51:58.781428Z
S1A_IW_SLC__1SDV_20250923T005158_20250923T005225_061108_079DF5_9C00.SAFE — 2025-09-23T00:51:58.181472Z
S1A_IW_ETA__AXDV_20250923T005158_20250923T005225_061108_079DF5_6E41.SAFE — 2025-09-23T00:51:58.123167Z
S1A_IW_RAW__0SDV_20250923T005155_20250923T005227_061108_079DF5_6251.SAFE — 2025-09-23T00:51:55.081627Z
S1A_IW_RAW__0SDV_20250923T005130_20250923T005202_061108_079DF5_F416.SAFE — 2025-09-23T00:51:30.081261Z
S1A_IW_GRDH_1SDV_20250922T124755_20250922T124820_061101_079DB9_1A15_COG.SAFE — 2025-09-22T12:47:55.368644Z
S1A_IW_GRDH_1SDV_20250922T124755_20250922T124820_061101_079DB9_6113.SAFE — 2025-09-22T12:47:55.368644Z
S1A_IW_SLC__1SDV_20250922T124754_202509

In [15]:
# Filter only Interferometric Wide Ground Range Detected (IW_GRDH) products
grdh_products = [p for p in results if "IW_GRDH" in p["Name"]]

print(f"\nFound {len(grdh_products)} IW_GRDH products.\n")

if not grdh_products:
    print("No IW_GRDH products found for this AOI and date range.")
else:
    for i, r in enumerate(grdh_products, start=1):
        print(f"{i}. {r['Name']} — {r['ContentDate']['Start']}")



Found 8 IW_GRDH products.

1. S1A_IW_GRDH_1SDV_20250923T005158_20250923T005223_061108_079DF5_7688_COG.SAFE — 2025-09-23T00:51:58.781428Z
2. S1A_IW_GRDH_1SDV_20250923T005158_20250923T005223_061108_079DF5_250B.SAFE — 2025-09-23T00:51:58.781428Z
3. S1A_IW_GRDH_1SDV_20250922T124755_20250922T124820_061101_079DB9_1A15_COG.SAFE — 2025-09-22T12:47:55.368644Z
4. S1A_IW_GRDH_1SDV_20250922T124755_20250922T124820_061101_079DB9_6113.SAFE — 2025-09-22T12:47:55.368644Z
5. S1A_IW_GRDH_1SDV_20250922T124730_20250922T124755_061101_079DB9_05C1.SAFE — 2025-09-22T12:47:30.368372Z
6. S1A_IW_GRDH_1SDV_20250922T124730_20250922T124755_061101_079DB9_06C8_COG.SAFE — 2025-09-22T12:47:30.368372Z
7. S1A_IW_GRDH_1SDV_20250918T004352_20250918T004417_061035_079B11_FD56.SAFE — 2025-09-18T00:43:52.415578Z
8. S1A_IW_GRDH_1SDV_20250918T004352_20250918T004417_061035_079B11_F39D_COG.SAFE — 2025-09-18T00:43:52.415578Z


In [16]:
# Allow the user to select a product and download it
if grdh_products:
    try:
        choice = int(input("\nEnter the number of the product you want to download: ").strip())
        if 1 <= choice <= len(grdh_products):
            selected = grdh_products[choice - 1]
            product_id = selected["Id"]

            # Construct the direct download URL
            download_url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"
            print(f"\nDownload URL:\n{download_url}\n")

            confirm = input("Do you want to download this file? (y/n): ").lower()
            if confirm == "y":
                filename = f"{selected['Name']}.zip"
                print(f"Downloading {filename}...")

                with requests.get(download_url, headers=headers, stream=True) as file_resp:
                    file_resp.raise_for_status()
                    with open(filename, "wb") as f:
                        for chunk in file_resp.iter_content(chunk_size=8192):
                            f.write(chunk)

                print(f"Download complete: {filename}")
            else:
                print("Download cancelled.")
        else:
            print("Invalid selection.")
    except ValueError:
        print("Please enter a valid number.")


KeyboardInterrupt: Interrupted by user